In [77]:
import pandas as pd
import numpy as np
import os, sys, json, pickle, time
from collections import Counter

In [25]:
#Some code to make it pickle files
years = range(1880, 2011)
data_dir = 'data/features_csv/test/'
data_all = pd.DataFrame()
files = os.listdir(data_dir)
num_files = len(files)
x = 0
for i in os.listdir(data_dir):
  if 'csv' not in i:
    continue
  
  sys.stdout.write("\r {:.2f} %".format(100*x/num_files))
  sys.stdout.flush()
  frame = pd.read_csv(os.path.join(data_dir,i), header = None)

  data_all = data_all.append(frame, ignore_index=True)
  x += 1
 
data = data_all.values
with open('data/512_test.pickle','wb') as f:
  pickle.dump(data, f)

 92.31 %

In [28]:
with open('data/512_train.pickle','rb') as f:
  train_data = pickle.load(f)
  
with open('data/512_test.pickle','rb') as f:
  test_data = pickle.load(f)
  
with open('data/512_validation.pickle','rb') as f:
  validation_data = pickle.load(f)

In [29]:
Xtrain, ytrain = train_data[:,1:-1], train_data[:,-1]
Xvalidation, yvalidation = validation_data[:,1:-1], validation_data[:,-1]
Xtest = test_data[:,1:]

In [79]:
test_id = [int(i) for i in test_data[:,0]]

# Forward feature selection
---

Let first try what they did in the exercises. So forward feature selection:
Lets (as in the exercise) try to find 5 features that are "good" using a random sample of 10000.

---

In [ ]:
#numpy.random.seed(1)
rnd_list = np.random.randint(0, train_data.shape[0],10000)
rnd_trn, rnd_val = rnd_list[:8000], rnd_list[8000:]

Xtrn, ytrn = Xtrain[rnd_trn], ytrain[rnd_trn]
Xval, yval = Xtrain[rnd_val], ytrain[rnd_val]

In [ ]:
start = time.time()

good_features = []
val_scores = []

feature_list = list(range(Xtrain.shape[1]))

for j in range(1,6):
    scores = []
    for i in feature_list:
        feature_slice = good_features+[i]
        X = Xtrn[:,feature_slice]
        model = KNeighborsClassifier(n_neighbors=10)
        model.fit(X, ytrn)
        scores.append(1- model.score(Xval[:,feature_slice],yval))
    
    best_ind = np.argmin(scores)
    best_feature = feature_list[best_ind]
    
    val_scores.append(scores[best_ind])
    good_features.append(best_feature)
    del feature_list[best_ind]
    print(good_features)
    
print('Seconds to run: {}'.format(time.time() - start))

In [ ]:
val_scores

### Conclusion

It seems that 5 features is not enough to get a proper score. maybe 10000 is also not enough to do this. 
Also the exercise data had 54 features, we have 256, so maybe crank up the number of features desired. 

---
# PCA

Also to furter reduce the number of features, lets look at PCA

----

In [ ]:
from sklearn.decomposition import PCA

pca= PCA()
pca.fit(Xtrain)

In [ ]:
print(pca.explained_variance_ratio_.cumsum()[65])
np.where(pca.explained_variance_ratio_.cumsum()> 0.90)

About 85% of the variance in data can be expressed using only 44 principle components. This seems like a nice number.

So lets transform the data and run a couple classifiers.

In [ ]:
TXtrain = pca.transform(Xtrain)[:,:65]
TXvalidation = pca.transform(Xvalidation)[:,:65]

In [12]:
# K nearest neighbor classifier
# Score is the mean accuracy

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 15)
knn.fit(Xtrain,ytrain)
knn.score(Xvalidation, yvalidation)

0.329370740212395

In [13]:
# Random Forest Classifier
# Score is the mean accuracy

from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators = 10)
rfc.fit(Xtrain, ytrain)
rfc.score(Xvalidation,yvalidation)

0.20399429386590584

# Multilayer Perceptron Classifier



In [10]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier((300,200), verbose =True, max_iter = 200, warm_start = True)

In [11]:
# A multilayer Preceptron
# Score is the mean accuracy again

mlp.fit(Xtrain, ytrain)
print('{} (Validation), {} (Train)'.format(mlp.score(Xvalidation,yvalidation), mlp.score(Xtrain, ytrain)))


Iteration 1, loss = 3.02373672
Iteration 2, loss = 2.51065148
Iteration 3, loss = 2.35484323
Iteration 4, loss = 2.25912217
Iteration 5, loss = 2.18263533
Iteration 6, loss = 2.12385648
Iteration 7, loss = 2.07159256
Iteration 8, loss = 2.02399032
Iteration 9, loss = 1.98631843
Iteration 10, loss = 1.94903998
Iteration 11, loss = 1.91207602
Iteration 12, loss = 1.88100177
Iteration 13, loss = 1.85244606
Iteration 14, loss = 1.82434157
Iteration 15, loss = 1.79970239
Iteration 16, loss = 1.77368930
Iteration 17, loss = 1.75118753
Iteration 18, loss = 1.73009354
Iteration 19, loss = 1.71154910
Iteration 20, loss = 1.69055406
Iteration 21, loss = 1.67183322
Iteration 22, loss = 1.65705165
Iteration 23, loss = 1.63721227
Iteration 24, loss = 1.62193730
Iteration 25, loss = 1.60747303
Iteration 26, loss = 1.59172125
Iteration 27, loss = 1.57487736
Iteration 28, loss = 1.56471175
Iteration 29, loss = 1.55179197
Iteration 30, loss = 1.53871677
Iteration 31, loss = 1.52751260
Iteration 32, los

/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


0.4037089871611983 (Validation), 0.6364487877983671 (Train)


In [60]:
with open('mlp.pickle','wb') as f:
  pickle.dump(mlp,f)
  
prediction = mlp.predict(Xtest)

In [80]:
all = set(range(1,12801))
missing = list(all - set(test_id))
rand_label = np.random.randint(1,125, len(missing))

pred = [int(i) for i in prediction]
pred.extend(rand_label)
test_id.extend(missing)

In [81]:
answers = pd.DataFrame(columns = ['id','predicted'])
answers['predicted'] = pred
answers['id'] = test_id

In [ ]:
answers.to_csv('512_prediction.csv', index = False)